In [2]:
from PIL import Image 
import random
from numpy import array 
import numpy as np
img = Image.open('Lena.png')

ar = array(img)

In [3]:
ar

array([[160, 160, 160, ..., 169, 154, 129],
       [161, 160, 159, ..., 168, 154, 129],
       [160, 161, 160, ..., 168, 154, 129],
       ...,
       [ 50,  49,  55, ..., 106, 103, 104],
       [ 49,  50,  58, ..., 107, 109, 111],
       [ 50,  49,  59, ..., 107, 108, 112]], dtype=uint8)

In [4]:
ar2 = np.transpose(ar)

In [5]:
print(ar2)

[[160 161 160 ...  50  49  50]
 [160 160 161 ...  49  50  49]
 [160 159 160 ...  55  58  59]
 ...
 [169 168 168 ... 106 107 107]
 [154 154 154 ... 103 109 108]
 [129 129 129 ... 104 111 112]]


In [6]:
def shuffle_arr(arr):
    row = len(arr)
    col = len(arr[0])
    for i in range(0,row):
        for j in range(0,col):
            arr[i][j] = ((row*row)+i+1)%256
    for i in range(0,int(row/4)):
        for j in range(int(col/4)):
            arr[i][j] = (row*row + 1)%256 - arr[i][j]
    for i in range(0, int(row/4)):
        for j in range(3*int(col/3),col):
            arr[i][j] = ((row*row)+1)%256-arr[i][j]
    for i in range(3*(int(row/4)),row):
        for i in range(0,int(col/4)):
            arr[i][j] = (row*row+1)%256 - arr[i][j]
    for i in range(int(row/4),3*int(row/4)):
        for j in range(int(col/4),3*int(col/4)):
            arr[i][j] = (row*row+1)%256 -arr[i][j]
    
    temp = np.array(arr)

    for i in range(0,row):
        for j in range(0,col):
            rowindex = int((arr[i][j]-1)%row)
            colindex = int((arr[i][j]-1)/col)
            temp[i][j] = arr[rowindex][colindex]

    return temp

In [7]:
ar3 = shuffle_arr(ar2)

In [8]:
ar3

array([[  0,   0,   0, ...,   0,   0,   0],
       [255, 255, 255, ..., 255, 255, 255],
       [254, 254, 254, ..., 254, 254, 254],
       ...,
       [254, 254, 254, ..., 254, 254, 254],
       [255, 255, 255, ..., 255, 255, 255],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [9]:
shuffled_img3 = Image.fromarray(ar3)

In [10]:
shuffled_img3.save('shuffeled_image3.png')

In [11]:
def key_gen(r,c):
    R = 3.99
    random_float = random.uniform(0,1)
    x = 0.4+(random_float/5)
    key = np.zeros((r,c))
    for i in range(0,r):
        for j in range(0,c):
            x = x*R*(1-x)
            key[i][j] = x*255
    
    return key

In [12]:
r = len(ar3)
c = len(ar3[0])
best_key = key_gen(r,c)
print(best_key)

[[249.0399228   23.22488505  84.22734159 ... 225.2855565  104.74508787
  246.26067159]
 [ 33.67486275 116.61899428 252.50994636 ... 136.3044472  253.14956675
    7.32965153]
 [ 28.40469027 100.71025358 243.13275432 ... 251.34485502  14.37498243
   54.1228698 ]
 ...
 [218.2747526  125.42986359 254.29544508 ... 237.54468009  64.87925234
  193.00478012]
 [187.2229074  198.55216896 175.36972059 ...  79.86229706 218.85383546
  123.77960676]
 [254.14592396   3.39634972  13.37094356 ... 176.87166787 216.22183045
  131.19568761]]


In [13]:
def encrypt(ar3,key):
    r = len(ar3)
    c = len(ar3[0])
    for i in range(0,r):
        for j in range(0,c):
            ar3[i][j] = int(ar3[i][j]) ^ int(key[i][j])
            
    cipher_img = np.array(ar3)
    for itr in range(0,int(r/16)):
        for i in range(0,r):
            for j in range(0,c):
                xmap = int((i+j)%r)
                ymap = int((i+2*j)%c)
                cipher_img[i][j] = cipher_img[xmap][ymap]
                
    
    
    return cipher_img

In [14]:
encrypted_img_arr = encrypt(ar3,best_key)
print(encrypted_img_arr)

[[249 100 160 ...  39 109  52]
 [156 121 111 ...  77  79  52]
 [148 176  17 ...  59 109 156]
 ...
 [ 53  18 223 ...  66 105 126]
 [141  43 160 ... 118  10 212]
 [222 100 175 ... 146  42 118]]


In [15]:
encrypted_img = Image.fromarray(encrypted_img_arr)

In [16]:
encrypted_img.save('encrypted_img.png')

In [17]:
def correlation_coefficient(img1, img2):
    # Ensure that the images have the same shape
    assert img1.shape == img2.shape, "The images have different shapes"
    
    # Calculate the mean of the images
    mean1 = np.mean(img1)
    mean2 = np.mean(img2)
    
    # Calculate the standard deviation of the images
    std1 = np.std(img1)
    std2 = np.std(img2)
    
    # Subtract the mean from each pixel and divide by the standard deviation
    normalized_img1 = (img1 - mean1) / std1
    normalized_img2 = (img2 - mean2) / std2
    
    # Calculate the sum of the product of the corresponding pixels
    sum_of_products = np.sum(normalized_img1 * normalized_img2)
    
    # Calculate the correlation coefficient
    return sum_of_products / (img1.size - 1)

In [20]:
correlation_coefficient(ar,encrypted_img_arr)

0.0006540932847117972